# <center> LSTM

In [1]:
import os
os.chdir("..")

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
import torch
import torch.nn as nn

## data

In [3]:
data_path = "data"

In [4]:
X_train = np.load(os.path.join(data_path, "X_train.npy"))
X_test = np.load(os.path.join(data_path, "X_test.npy"))
y_train = np.load(os.path.join(data_path, "y_train.npy"))
y_test = np.load(os.path.join(data_path, "y_test.npy"))

In [5]:
X_train.shape, y_train.shape

((7352, 128, 9), (7352, 6))

In [6]:
X_test.shape, y_test.shape

((2947, 128, 9), (2947, 6))

In [5]:
y_train_num = y_train.argmax(1)
y_test_num = y_test.argmax(1)

#### посмотрим на баланс классов

In [6]:
pd.Series(y_train_num).value_counts().sort_index()

0    1226
1    1073
2     986
3    1286
4    1374
5    1407
dtype: int64

В первом приближении классы можно считать близкими к сбалансированным

#### нормализуем данные

In [7]:
X_train_mean = X_train.mean(0)
X_train_std = X_train.std(0)

In [9]:
X_train_norm = (X_train - X_train_mean) / X_train_std
X_test_norm = (X_test - X_train_mean) / X_train_std

#### model

In [12]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, num_layers=1, batch_first=True):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.batch_first = batch_first
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True)  #lstm
        self.fc_1 =  nn.Linear(hidden_size, 128)  #fully connected 1
        self.fc = nn.Linear(128, num_classes)  #fully connected last layer

        self.relu = nn.ReLU()
    
    def forward(self, input):
        out, _ = self.lstm(input)
        out = self.relu(out[:, -1, :])
        out = self.fc_1(out)  #first Dense
        out = self.relu(out)  #relu
        out = self.fc(out)  #Final Output
        return out